In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Q1 Training Datasets

Training Datasets (TDS) in EOTDL are categorized into different [quality levels](/docs/datasets/quality), which in turn will impact the range of functionality that will be available for each dataset.

In this post you will learn about Q1 datsets, datasets with STAC metadata. 

## Ingesting Q1 datasets

To ingest a Q1 datasets you will need its STAC metadata.

### Datasets with STAC metadata

Some datasets already have STAC metadata, and can be ingested directly into EOTDL. For example, the [AgriFieldNet Competition Dataset
](https://mlhub.earth/data/ref_agrifieldnet_competition_v1) dataset is available on Radiant MLHub. To follow with this tutorial, please download the dataset and extract it in the `data` folder.

In [5]:
import os 

os.listdir('data')

['ref_agrifieldnet_competition_v1_source',
 'ref_agrifieldnet_competition_v1_labels_test',
 'ref_agrifieldnet_competition_v1_labels_train',
 'ref_agrifieldnet_competition_v1']

You can use the eotdl library to ingest the dataset.

> Alternatively, you can use the CLI. Check the docs to learn more.

In [7]:
from eotdl.datasets import ingest_q1

ImportError: cannot import name 'ingest_q1' from 'eotdl.datasets' (/Users/sensio/miniconda3/lib/python3.9/site-packages/eotdl/datasets/__init__.py)

In [4]:
ingest_q1('agrifieldnet', 'data/ref_agrifieldnet_competition_v1/catalog.json')

holas


### Creating STAC metadata